# Link to model data

In [1]:
MODEL_A_GS_PATH = 'https://docs.google.com/spreadsheets/d/1KF1sLGA6D8sV6ckvpdsDrpVB9pfgPOSWW2qSzyG_zhQ/edit#gid=461085463'
MODEL_B_GS_PATH = 'https://docs.google.com/spreadsheets/d/1Xi1gQyf7hWSjmOvsf1My08YP5qaOM55D5kTfC0E_r2o/edit#gid=1989900327'
MAX_NUM_RECS = 20


# Support functions

In [2]:
# Support code
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

MODEL_A_GS_CSV_PATH = MODEL_A_GS_PATH.replace('/edit#gid=', '/export?format=csv&gid=')
MODEL_B_GS_CSV_PATH = MODEL_B_GS_PATH.replace('/edit#gid=', '/export?format=csv&gid=')

model_a_df = None
model_b_df = None

unique_values_project_features = {}
unique_values_manufacture = {}

all_widgets_dict = {}
out = widgets.Output()

def get_project_features(model_df): 
    model_columns = list(model_df.columns)
    return model_columns[:model_columns.index('separator')]

def get_ranking_columns(model_df):
    model_columns = list(model_df.columns)
    return model_columns[model_columns.index('separator')+1:]
    
    
    
def prepare_data(): 
    model_a_df = pd.read_csv(MODEL_A_GS_CSV_PATH).fillna('')
    model_b_df = pd.read_csv(MODEL_B_GS_CSV_PATH).fillna('')
    model_a_columns = list(model_a_df.columns)
    model_b_columns = list(model_b_df.columns)
    # Validate same columns
    if not set(model_a_columns) == set(model_b_columns):
        print("ERROR: models don't have the same features: ")
        print("Model A: " + str(set(model_a_columns)))
        print("Model B: " + str(set(model_b_columns)))
        return None
    
    project_features = get_project_features(model_a_df)

#     manufacturer_details = model_a_columns[model_a_columns.index('separator')+1:]
    for column in project_features:
        if set(model_a_df[column].unique()) != set(model_b_df[column].unique()):
            print("ERROR: column " + str(column) + " values in model A is different from model B")
            return None
        unique_values_project_features[column] = list(model_a_df[column].unique())
    unique_values_manufacture['manufacture_country'] = list(set(model_a_df['manufacture_country'].unique()).union(set(model_b_df['manufacture_country'].unique())))
    # print(unique_values_project_features)
    return model_a_df, model_b_df, unique_values_project_features

def get_recs():
#     criteria_idx = True
    criteria_idx_a = True
    criteria_idx_b = True
    for key in unique_values_project_features.keys():
#         print(str(key) + ' = ' + str(all_widgets_dict[key].value))
        criteria_idx_a = criteria_idx_a & (model_a_df[key] == all_widgets_dict[key].value)
        criteria_idx_b = criteria_idx_b & (model_b_df[key] == all_widgets_dict[key].value)
    
    criteria_idx_a = criteria_idx_a & model_a_df['manufacture_country'].isin(all_widgets_dict['manufacture_country'].value)
    criteria_idx_b = criteria_idx_b & model_b_df['manufacture_country'].isin(all_widgets_dict['manufacture_country'].value)
    for key in all_widgets_dict.keys():
        if hasattr(all_widgets_dict[key], 'value'):
            print(str(key) + ":" + str(all_widgets_dict[key].value))
    ret_a_df = model_a_df[criteria_idx_a][['manufacturer_name', 'post_id_manuf', 'manufacture_country', 'predBidProb']]
    ret_b_df = model_b_df[criteria_idx_b][['manufacturer_name', 'post_id_manuf', 'manufacture_country', 'predBidProb']]
    
    return ret_a_df.head(MAX_NUM_RECS), ret_b_df.head(MAX_NUM_RECS)
    
def on_get_recs_button_clicked(b):
    with out:
        clear_output()
        ret_a_df, ret_b_df = get_recs()
        print("")
        print("Model A: ")
        display(ret_a_df)
        print("")
        print("Model B: ")
        display(ret_b_df)
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Last button click: {}.".format(current_time))

def open_view():    
    all_widgets_dict = {}
    for key in unique_values_project_features.keys():   
        all_widgets_dict[key] = widgets.Dropdown(
            style={'description_width': 'initial'},
            options=unique_values_project_features[key],
            value=unique_values_project_features[key][0],
            description=key,
            disabled=False,
        )
        all_widgets_dict['manufacture_country'] = widgets.SelectMultiple(
            options=unique_values_manufacture['manufacture_country'],
            value=[],
            #rows=10,
            description='manufacture_country',
            disabled=False, 
            style = {'description_width': '140px'}
        )

        
        
    all_widgets_dict['get_recs_button'] = widgets.Button(description="Get recs")
    all_widgets_dict['get_recs_button'].on_click(on_get_recs_button_clicked)
    all_widgets = widgets.VBox(list(all_widgets_dict.values()))
    return all_widgets_dict, all_widgets

def show_get_recs_button(all_recs_button):
    display(out)
    display(all_recs_button)

## Read CSV 

In [3]:
model_a_df, model_b_df, unique_values_project_features = prepare_data()

# Get recommendations

In [4]:
all_widgets_dict, all_widgets = open_view()
all_widgets

In [5]:
display(out)

Output()